In [ ]:
# 사용할 모듈 import

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
# 쿠다 GPU를 사용할 수 있으면 사용할 변수 생성

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(1)
if device == "cuda" :
    torch.cuda.manual_seed_all(1)

In [ ]:
# 학습 하기위한, 설정 변수 생성

lr = 0.001
epochs = 15
batch_size = 100

In [ ]:
# 사용할 데이터를, train과 test로 나눠서 가져옴

mnist_train = dsets.MNIST(
    root="/content/drive/MyDrive/Study/00_pytorch/data",
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
mnist_test = dsets.MNIST(
    root="/content/drive/MyDrive/Study/00_pytorch/data",
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [ ]:
# 학습하기 전, 데이터의 배치사이즈를 조정 한다는 등에 필요한 DataLoader로 감싸주면서, 설정할거 설정.

data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [ ]:
# 학습할 모델의 구조 생성

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )

        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x) # 첫번째 cnn층  (con연산 > 활성화 함수 > pool)
        out = self.layer2(out) # 두번째 cnn층
        out = out.view(out.size(0), -1) # cnn층에서 나온 결과를 fully conect하기 위해서 데이터 모양 변환
        out = self.fc(out)
        return out

In [ ]:
# 모델 인스턴트화

model = CNN().to(device)

In [ ]:
# 모델을 최적화 하기 위한, 손실함수와, 최적화 함수 구현.

loss_funcion = torch.nn.CrossEntropyLoss().to(device) # 소프트맥스 함수 포함되있음.
optimizer = torch.optim.Adam(model.parameters(), lr)
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)


In [ ]:
# 한번에 처리할 데이터 수 보기.  (총 데이터 / 배치사이즈)

total_batch = len(data_loader)
print("총 배치 수 : ", total_batch)

총 배치 수 :  600


In [ ]:
# test

# for x,y in data_loader:
#     print(x)
#     print(y)
#     break

In [ ]:
# 앞서 구현한 모델에 대해서, 실제적으로 학습을 하는 구문.

for epoch in range(epochs): # epochs 몇번 학습할 것인가.
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss_funcion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
    print("{} : {}".format(epoch, avg_cost))

0 : 0.21592819690704346
1 : 0.06004781648516655
2 : 0.04463717341423035
3 : 0.035597264766693115
4 : 0.02964676544070244
5 : 0.025230297818779945
6 : 0.021439606323838234
7 : 0.017819155007600784
8 : 0.015195440500974655
9 : 0.012972123920917511
10 : 0.011133020743727684
11 : 0.009545430541038513
12 : 0.0074052708223462105
13 : 0.006663447245955467
14 : 0.007039204705506563


In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    print("X_test : ", X_test.shape)
    predict = model(X_test) # 학습된 모델에 데이터 집어넣음
    print("predict : ", predict)
    correct_prediction = torch.argmax(predict, 1) == Y_test
    print("correct_prediction : ", correct_prediction)
    accuracy = correct_prediction.float().mean()
    print("Accuracy : ", accuracy.item())

X_test :  torch.Size([10000, 1, 28, 28])
predict :  tensor([[-7664.9365, -2286.8684, -2631.0149,  ...,  4750.6118, -2894.0317,
            40.8609],
        [-3169.8486, -1643.7675,  4516.6592,  ..., -4977.1494, -2233.8491,
         -3281.3982],
        [-4476.8511,  3368.1487, -2282.2522,  ...,   -10.0839,   -94.4707,
         -2898.0481],
        ...,
        [-9060.1836, -2851.5061, -6996.4497,  ..., -1208.5099, -1545.7174,
          -656.4276],
        [-2610.1997, -8855.5312, -3991.2188,  ..., -5674.1797,   954.8950,
         -3989.2744],
        [-3730.1318, -8391.1875, -3707.4475,  ..., -7058.7383, -2648.4312,
         -4054.4377]], device='cuda:0')
correct_prediction :  tensor([True, True, True,  ..., True, True, True], device='cuda:0')
Accuracy :  0.9888999462127686


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
